#
**PyTorch Tutorial: 01 Tensors**
**Overview**
In this series of tutorials, we will introduce some basics of PyTorch. Pay attention that the API's of any library will be subject to changes, therefore it is very important to check the offcial documentation before proceeding.

We will cover the following materials in this tutorial.

1.Creating tensors.
2.Some common manipulations (including broadcast and einsum).
**Creating tensors**

In [2]:
import torch

In [3]:
my_tensor = torch.tensor([[0.0, 1.0],[0.1, 0.2]])
my_tensor

tensor([[0.0000, 1.0000],
        [0.1000, 0.2000]])

In [4]:
new_tensor= my_tensor.int().float()
new_tensor

tensor([[0., 1.],
        [0., 0.]])

In [8]:
import numpy as np
np_tensor = np.array([[0.1,1.0],[1.0,0.2]])


numpy数组转换为tensor numpy默认的浮点数是64位双精度浮点数（GPU 需要单精度）

In [12]:
tensor_from_np = torch.tensor(np_tensor)
tensor_from_np

tensor([[0.1000, 1.0000],
        [1.0000, 0.2000]], dtype=torch.float64)

torch.tensor数组转换为numpy 

In [14]:
to_numpy = my_tensor.numpy()
to_numpy

array([[0. , 1. ],
       [0.1, 0.2]], dtype=float32)

tensor存到GPU中

In [16]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
my_tensor.to(device=device) # See https://pytorch.org/docs/stable/notes/cuda.html for complete examples

tensor([[0.0000, 1.0000],
        [0.1000, 0.2000]])

In [17]:
to_numpy = my_tensor.numpy()
to_numpy

array([[0. , 1. ],
       [0.1, 0.2]], dtype=float32)

提取tensor{[0]}中的数据

In [18]:
single_number =  torch.tensor([0])
single_number

tensor([0])

In [19]:
single_number.item()

0

tensor中的Autograd，自动求解梯度的功能
requires_grad=True 要求计算梯度
pow(2).sum() 二次方后求整数
backward() 把对应的tensor的梯度都记录下来

In [20]:

tensor_with_gradient = torch.tensor([[0.1,1.0], [1.0, 2.0]], requires_grad=True)
result =tensor_with_gradient.pow(2).sum()
result.backward()
tensor_with_gradient.grad

tensor([[0.2000, 2.0000],
        [2.0000, 4.0000]])

In [21]:
tensor_with_gradient.detach_() # This will make sure that the Tensor will never need gradient


tensor([[0.1000, 1.0000],
        [1.0000, 2.0000]])

**Basic Operations**

In [22]:
x= torch.tensor([[0.1,1.0],[2.0,1.0]])
x+1

tensor([[1.1000, 2.0000],
        [3.0000, 2.0000]])

In [23]:
x*2

tensor([[0.2000, 2.0000],
        [4.0000, 2.0000]])

In [24]:
y= torch.tensor([[0.1,2.0],[2.0,3.0]])
x+y

tensor([[0.2000, 3.0000],
        [4.0000, 4.0000]])

python中的slice  
x[:,:] 要所有的位数

In [25]:
x[:,:]

tensor([[0.1000, 1.0000],
        [2.0000, 1.0000]])

x[1,:]  第一维只要一，第二维要全部

In [26]:
x[1,:]

tensor([2., 1.])

x[:,1] 第二维的第一个维度，第一维要所有的维度

In [27]:
x[:,1] 

tensor([1., 1.])

squeeze一个维度上只有一位的时候，去掉这个维度  unsqueeze增加一个只有一的维度
shape 查看维度

In [29]:
x = x.unsqueeze(0)
x

tensor([[[[0.1000, 1.0000],
          [2.0000, 1.0000]]]])

In [30]:
x.shape

torch.Size([1, 1, 2, 2])

In [31]:
x = x.squeeze(0)
x

tensor([[[0.1000, 1.0000],
         [2.0000, 1.0000]]])

In [32]:
x.shape

torch.Size([1, 2, 2])

In [38]:
y.shape

torch.Size([2, 2])

算一个线性的层，从矩阵的角度，让W是它的权重，X 是它的输入，W乘以X或者X乘以W  w为10*10 X为10*1 
假设X是一个batch进来的，有16个观测，16*10*1  背后复制

Two tensors are “broadcastable” if the following rules hold:

Each tensor has at least one dimension.（至少有一个维度）

When iterating over the dimension sizes, starting at the trailing dimension, the dimension sizes must either be equal, one of them is 1, or one of them does not exist.

 x=torch.empty(5,7,3)
 
 y=torch.empty(5,7,3)
 
same shapes are always broadcastable (i.e. the above rules always hold)

x=torch.empty((0,))

 y=torch.empty(2,2)
 
 x and y are not broadcastable, because x does not have at least 1 dimension

can line up trailing dimensions

 x=torch.empty(5,3,4,1)
 
 y=torch.empty(  3,1,1)
 
 x and y are broadcastable.
 
 1st trailing dimension: both have size 1
 
 2nd trailing dimension: y has size 1
 
 3rd trailing dimension: x size == y size
 
 4th trailing dimension: y dimension doesn't exist
 

but:

 x=torch.empty(5,2,4,1)
 
 y=torch.empty(  3,1,1)
 
 x and y are not broadcastable, because in the 3rd trailing dimension 2 != 3

In [34]:
z = x + y 
z # This is a case of broadcast, see https://pytorch.org/docs/stable/notes/broadcasting.html for details

tensor([[[0.2000, 3.0000],
         [4.0000, 4.0000]]])

x ([1, 2, 2]) y([2, 2]) 结果为([1, 2, 2])

In [36]:
z.shape

torch.Size([1, 2, 2])

In [37]:
z= z.squeeze()
z

tensor([[0.2000, 3.0000],
        [4.0000, 4.0000]])

Einsum 爱因斯坦发明的一个记载矩阵或者向量乘和加法的一个非常简便的记录方式

目的：在这种计算时，没有办法写成标准的内置矩阵运算的时候，可以比较方便做这件事

Output: O的ij位 等于X下标为i乘以y的下标为j（笔记中）

In [39]:
x= torch.randn(5)
y= torch.randn(5)
torch.einsum('i,j->ij',x,y)

tensor([[ 1.7042,  2.7819,  0.1232, -1.1915, -1.1433],
        [ 1.6281,  2.6578,  0.1177, -1.1383, -1.0922],
        [ 0.1894,  0.3092,  0.0137, -0.1324, -0.1271],
        [-0.3016, -0.4924, -0.0218,  0.2109,  0.2023],
        [ 0.8640,  1.4104,  0.0625, -0.6041, -0.5796]])

In [40]:
A = torch.randn(3,5,4)
l = torch.randn(2,5)
r = torch.randn(2,4)
torch.einsum('bn,anm,bm->ba', l, A, r)

tensor([[ 5.2664,  1.9942, -5.3592],
        [ 2.6677, -5.0950, -6.2886]])